In [1]:
import os
import pdfplumber
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")


# Replace 'folder_path' with the path to your folder containing PDF files--its local
folder_path = 'dec2024'

data_dicts = []


for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_file_path = os.path.join(folder_path, filename)
        invoice = filename.split('.pdf')

        with pdfplumber.open(pdf_file_path) as pdf:
            
            for page_number in range(len(pdf.pages)):
             
                page = pdf.pages[page_number]

              
                page_text = page.extract_text()

                data = re.findall(r'Connected Load\s*:\s*(\d+\.\d+)[\s\S]*?Outstanding amount\(Rs.\)#([\d.]+)',
                                  page_text)
                if data:
                    consumer_data = data[0]
                else:
                    consumer_data = ("", "")

                data1 = re.findall(r'ConsumerId\s*:\s*(\d+)[\s\S]*?', page_text)
                if data1:
                    consumer_data1 = data1[0]
                else:
                    consumer_data1 = ""

                data2 = re.findall(r'LPSC Charge (\d+\.\d+)[\s\S]*?', page_text)
                if data2:
                    consumer_data2 = data2[0]
                else:
                    consumer_data2 = "0"
                data3 = re.findall(r'Government Subsidy (-\d+\.\d+)[\s\S]*?', page_text)
                if data3:
                    consumer_data3 = data3[0]
                else:
                    consumer_data3 = "0"
                data4 = re.findall(r'Total Charges \(Rs\) ([0-9.]+)[\s\S]*?', page_text)
                if data4:
                    consumer_data4 = data4[0]
                else:
                    consumer_data4 = "0"

                data5 = re.findall(r'Installation No\s*:\s*(\d+)[\s\S]*?', page_text)
                if data5:
                    consumer_data5 = data5[0]
                else:
                    consumer_data5 = "NA"
                data6 = re.findall(r'(?i)Bill Month\s*:\s*([A-Z][a-z]+[,.\s]*\d{4})', page_text)
                if data6:
                    consumer_data6 = data6[0]
                else:
                    consumer_data6 = "NA"

                
                meter_reading_pattern = r'Meter No Time Previous Present Multiplying Unit Max Demand\s+Reading ' \
                                         r'Reading Factor\(MF\) consumed \(KVA\)\s+([A-Z0-9]+)\s+[A-Z]+ ' \
                                         r'([0-9.]+) ([0-9.]+) ([0-9.]+) ([0-9.]+)'
                matches = re.findall(meter_reading_pattern, page_text)
                for match in matches:
                    meter_no, previous_reading, present_reading, multi_fact, units_cons = match

                    if units_cons == '0':
                        pattern = r'Fixed/Demand Charge ([\d.]+)[\s\S]*?Total Charges \(Rs\) ([0-9.]+)[\s\S]*?' \
                                  r'Rental Charge\s*([\d.]+)'
                        charges_data = re.findall(pattern, page_text)
                        if charges_data:
                            charges_data = charges_data[0]
                        else:
                            charges_data = ("", "", "")

                        data_dicts.append({
                            "InvoiceNo": invoice[0],
                            "Installation_No": consumer_data5,
                            "ConsumerId": consumer_data1,
                            "Bill_Month": consumer_data6,
                            "Connected_Load": consumer_data[0],
                            "Outstanding_amount": consumer_data[1],
                            "Meter_No": meter_no,
                            "Previous_Reading": previous_reading,
                            "Present_Reading": present_reading,
                            "MF": multi_fact,
                            "Units_Cons": units_cons,
                            "Fixed/Demand Charge": charges_data[0],
                            "Rental_Charge": charges_data[2],
                            "LPSC": consumer_data2,
                            "Govt Subsidy": consumer_data3,
                            "Total_Charge": consumer_data4
                        })

                    else:
                        pattern2 = r'Energy Charge ([\d.]+)[\s\S]*?Total Charges \(Rs\) ([0-9.]+)[\s\S]*?' \
                                   r'Fixed/Demand Charge ([\d.]+)[\s\S]*?Rental Charge\s*([\d.]+)'
                        charges_data2 = re.findall(pattern2, page_text)
                        if charges_data2:
                            charges_data2 = charges_data2[0]
                        else:
                            charges_data2 = ("", "", "", "")

                        data_dicts.append({
                            "InvoiceNo": invoice[0],
                            "Installation_No": consumer_data5,
                            "ConsumerId": consumer_data1,
                            "Bill_Month": consumer_data6,
                            "Connected_Load": consumer_data[0],
                            "Outstanding_amount": consumer_data[1],
                            "Meter_No": meter_no,
                            "Previous_Reading": previous_reading,
                            "Present_Reading": present_reading,
                            "MF": multi_fact,
                            "Units_Cons": units_cons,
                            "EnergyCharge": charges_data2[0],
                            "Fixed/Demand Charge2": charges_data2[2],
                            "Rental_Charge": charges_data2[3],
                            "LPSC": consumer_data2,
                            "Govt Subsidy": consumer_data3,
                            "Total_Charge": consumer_data4
                        })

combined_df = pd.DataFrame(data_dicts)

numeric_columns = ['InvoiceNo', 'Installation_No', 'ConsumerId',
                   'Connected_Load', 'Outstanding_amount', 'Previous_Reading',
                   'Present_Reading', 'MF', 'Units_Cons', 'Fixed/Demand Charge',
                   'Rental_Charge', 'LPSC', 'Govt Subsidy', 'Total_Charge', 'EnergyCharge',
                   'Fixed/Demand Charge2']
combined_df[numeric_columns] = combined_df[numeric_columns].apply(pd.to_numeric, errors="coerce")
df=combined_df.copy()
df['Fixed/Demand Charge']=df['Fixed/Demand Charge'].fillna(df['Fixed/Demand Charge2'])
df=df.drop(columns=['Fixed/Demand Charge2'])
df['Rental_Charge']=df['Rental_Charge'].fillna(25.0)

new_dict={213127978: "Natukjaykrishnapur", 213127933: "Beurgram", 213127980: "Chaksadi",
213069106: "Joynagar", 213127979: "Lakshmanpur", 213127981: "Dihrgagram",
213073810: "Shyamsundarpur", 213073812: "Radhakantapur", 213073813: "Barasimulia",
213073814: "Isabpur", 213073811: "Dehipalsa", 213039142: "Guchati",
213107071: "Mohanpur", 213107072: "Raniganj", 213107069: "Jakrasirsha",
213103031: "Mamudpur", 213103032: "Tenthua", 212009260: "Tangaguria",
212134540: "Dasagram", 212134538: "Sabong", 212130389: "Jorhut",
212143465: "Maligram", 212144160: "Naya",  212130376: "Bhusulia",
212144161: "Tangur", 212129733: "Angua", 212129740: "Bejda",
212140614: "Zinsahar",  212172076: "Kusumda",  212172077: "Rampura",
212172085: "Sautia", 212009809: "Karkoda", 214012745: "Kantapal",
200325904: "Kespur", 212140603: "Mukshedpur", 213229548: "Narayanchak",
212009684: "Palaskhanda",222247278: "Sibracharmanpur",
200470432: "Kuldiha", 213017470: "Patharghata", 203055483: "Renjura",
212140730: "Shaymsundarpur", 212130380: "Makorda", 202809084: "Rambhadrapur",
212140715: "Jamda"
}

df['scheme'] = df['ConsumerId'].map(new_dict)

filename='df_excel_21_july24.xlsx'
df.to_excel(filename)



In [2]:
df.head()

,InvoiceNo,Installation_No,ConsumerId,Bill_Month,Connected_Load,Outstanding_amount,Meter_No,Previous_Reading,Present_Reading,MF,Units_Cons,EnergyCharge,Rental_Charge,LPSC,Govt Subsidy,Total_Charge,Fixed/Demand Charge,scheme
0,400033236670,8078425,212143465,"DEC,2024",23.53,2811126.75,ST511243,912809.0,913701.0,1.0,892.0,NaN,25.0,0,0.0,5568.52,NaN,Maligram
1,400033236671,8078427,212144160,"DEC,2024",29.41,3498702.51,ST708996,589278.0,590421.0,1.0,1143.0,NaN,25.0,0,0.0,7090.98,NaN,Naya
2,402025147128,8036069,212134540,"DEC,2024",29.41,2890039.81,XB000099,4212.0,8413.0,1.0,4201.0,19576.66,25.0,0,0.0,21342.41,1740.75,Dasagram
3,402025147155,6307351,200325904,"DEC,2024",35.11,766567.60,ST602461,106533.0,108277.0,1.0,1744.0,NaN,25.0,0,0.0,10131.71,NaN,Kespur
4,402025147162,6341274,212140603,"DEC,2024",23.53,161334.18,GX077822,6999.0,7300.0,1.0,301.0,1552.45,25.0,0,0.0,2473.11,895.66,Mukshedpur
